In [1]:
import random
import numpy as np
import pandas as pd
import re
import spacy
from get_object_path import get_object_path

nlp = spacy.load("en_core_web_md")

In [2]:
# !python -m spacy download en_core_web_md 

In [3]:
df = pd.read_csv('test.csv')

### Get WebUI function with objects as arguments

In [4]:
# # Get fun_obj_dict
# def update_func_obj_dict(script, func_obj_dict):
#     lines = script.split('\n')
#     func_name_pattern = re.compile('WebUI\.([a-zA-Z]*)(.*)')
#     obj_pattern = re.compile('[^\']*findTestObject*\((\'([^\']*)\')+.*')
#     for line in lines:
#         result = func_name_pattern.search(line)
#         if result:
#             func_name = result.group(1)
#             obj_result = obj_pattern.search(result.group(2))
#             if obj_result:
#                 obj = obj_result.group(2)
#                 obj = '/'.join(obj.split('/')[-2:])
#                 if func_name in func_obj_dict:
#                     func_obj_dict[func_name].add(obj)
#                 else:
#                     obj_set = set()
#                     obj_set.add(obj)
#                     func_obj_dict[func_name] = obj_set
#         else:
#             continue

#     return func_obj_dict

In [5]:
# def get_func_obj_dict(df):
#     func_obj_dict = {}
#     for script in df['test_scripts']:
#         if script is np.NaN:
#             continue
#         func_obj_dict = update_func_obj_dict(script, func_obj_dict)
#     return func_obj_dict

In [6]:
# func_obj_dict = get_func_obj_dict(df)
# # func_obj_dict

### Revert back to scripts

In [7]:
func_verb_alignment = {
    'setText': {'enter', 'input'},
    'click': {'click'},
    'waitForElementPresent': {'wait'},
    'getText': {'get'},
    'verifyMatch': {'check'}
}

verb_func_alignment = {}
for func, verbs in func_verb_alignment.items():
    for verb in verbs:
        verb_func_alignment[verb] = func
verb_func_alignment

{'input': 'setText',
 'enter': 'setText',
 'click': 'click',
 'wait': 'waitForElementPresent',
 'get': 'getText',
 'check': 'verifyMatch'}

### Action_widgets

In [8]:
# path_dict = defaultdict(lambda: defaultdict(set))
# for values in func_obj_dict.values():
#     for value in values:
#         get_path_list = value.split('/')
#         path = '/'.join(get_path_list[0:-1])
#         get_element_list = get_path_list[-1].split('_')
#         element = get_element_list[1:]
#         if len(get_element_list) > 1:
#             element = '_'.join(get_element_list[1:])
#             path_dict[path][element].add(value)
#         else:
#             element = get_element_list[0]
#             path_dict[path][element].add(value)


# def page_str_modifier(page):
#     if len(page.split('/')) > 1:
#         page = ''.join(page.split('/')[1:])
#         page = ''.join(page.split('_'))
#     else:
#         page = ''.join(page.split('_'))[1:]
#     page = ' '.join(re.findall('[A-Z][a-z]+', page))
#     return page


# def location_modifier(location):
#     location = ' '.join(location.split('_'))
#     all_list = (re.findall('[A-Z]*[a-z]*', location))
#     starter = ''
#     for string in all_list:
#         if string == '':
#             continue
#         else:
#             starter += string + ' ' 
#     return starter.rstrip()

In [9]:
# action_widget = func_obj_dict.copy()
# action_widget['enter'] = action_widget.pop('setText')
# action_widget['input'] = action_widget['enter']

# action_widget['wait'] = set()
# for key in action_widget.keys():
#     if 'wait' in key.lower():
#         action_widget['wait'].update((action_widget[key]))

# action_widget['get'] = action_widget.pop('getText')

# alternate_dict = defaultdict(lambda: defaultdict(set))
# for key in path_dict.keys():
#     for values in path_dict[key].values():
#         for value in values:
#             key_list = value.split('/')[-1].split('_')
#             if len(key_list) == 1:
#                 key_list = key_list[0].split('-')
#             key_value = '_'.join(key_list[1:])
#             page_list = value.split('/')[0:-1]
#             page_value = '/'.join(page_list)
#             alternate_dict[location_modifier(key_value)][page_str_modifier(page_value)].add(value)

### Find obj path

In [10]:
# def get_page(page):
#     max_sim = -np.inf
#     max_key = ''
#     for key in path_dict.keys():
#         similarity = nlp(page).similarity(nlp(page_str_modifier(key)))
#         if similarity > max_sim:
#             max_sim = similarity
#             max_key = key
#     return max_key


# def get_location(page, location):
#     min_dist = np.inf
#     min_key = ''
#     for key in path_dict[page].keys():
#         distance = Levenshtein.distance(location, key.lower())
#         if distance < min_dist:
#             min_dist = distance
#             min_key = key
#     return min_key


# def get_alternate_location(location):
#     max_sim = -np.inf
#     max_key = ''
#     for key in alternate_dict.keys():
#         similarity = nlp(location.lower()).similarity(nlp(key.lower()))
#         if similarity > max_sim:
#             max_sim = similarity
#             max_key = key
#     return max_key


# def get_alternate_page(location, page):
#     max_sim = -np.inf
#     max_key = ''
#     for key in alternate_dict[location].keys():
#         similarity = nlp(page.lower()).similarity(nlp(key.lower()))
#         if similarity > max_sim:
#             max_sim = similarity
#             max_key = key
#     return max_key


# def get_obj_path(parsed_test_case):
#     page = get_page(parsed_test_case['page'])
#     if 'location' in parsed_test_case.keys():
#         location = get_location(page, parsed_test_case['location'])
#     else:
#         location = get_location(page, parsed_test_case['value'])
#     for element in path_dict[page][location]:
#         if element in action_widget[parsed_test_case['V']]:
# #             return element
#             return f'findTestObject(\'{element}\')'
#         else:
#             if 'location' in parsed_test_case.keys():
#                 location = get_alternate_location(parsed_test_case['location'])
#             else:
#                 location = get_alternate_location(parsed_test_case['value'])
#             page = get_alternate_page(location, parsed_test_case['page'])
#             for element in alternate_dict[location][page]:
#                 if element in action_widget[parsed_test_case['V']]:
# #                     return element
#                     return f'findTestObject(\'{element}\')'
#                 else:
#                     return 'Unidentified'

### Reformat output of object_path

In [11]:
def obj_path(test_case_dict):
    output = get_object_path(test_case_dict)

    if type(output) == 'set':
        print(output)
        return random.sample(output, 1)
    else:
        return f'findTestObject(\'{output}\')'

### Alignment

In [12]:
def align_one_line_to_script(test_case_dict):
    # Get aligned WebUI function
    action = test_case_dict['V']
    if action in verb_func_alignment:
        fun = verb_func_alignment[action]
    else:
        max_sim = -np.inf
        for verb in verb_func_alignment:
            similarity = nlp(action).similarity(nlp(verb))
            if similarity > max_sim:
                max_sim = similarity
                fun = verb_func_alignment[verb]

    # Get arguments for the WebUI function
    args = []

    if fun == 'setText':
        arg1 = obj_path(test_case_dict)
        arg2 = f"'{test_case_dict['value']}'"
        args = [arg1, arg2]
    if fun == 'click':
        arg1 = obj_path(test_case_dict)
        args = [arg1]
    if fun == 'waitForElementPresent':
        arg1 = obj_path(test_case_dict)
        args = [arg1, str(test_case_dict['time']), 'FailureHandling.STOP_ON_FAILURE']
    if fun == 'getText':
        args = [obj_path(test_case_dict)]
#     if fun == 'verifyMatch':
#         args = [test_case_dict['page'], test_case_dict['page'], 'false']

    script_dict = {"action": fun, "entities": args}

    # Get variable
    if fun == 'getText':
        obj = args[-1].split('\'')[-2].split('_')[-1]
        script_dict['var_name'] = 'actual' + obj
        script_dict['var_type'] = 'def'

    return script_dict

In [13]:
align_one_line_to_script({'V': 'input',
       'location': 'to email textbox',
       'value': 'invalid@wrong',
       'page': 'login'})

{'action': 'setText',
 'entities': ["findTestObject('Page_Login/tbx_Email')", "'invalid@wrong'"]}

### Revert to scripts

In [14]:
def revert_to_script_dicts(test_case):
    script_dicts = []
    for line_dict in test_case:
        if line_dict['V'] not in verb_func_alignment:
            continue
        script_dict = align_one_line_to_script(line_dict)
        script_dicts.append(script_dict)
    return script_dicts


def revert_to_script(script_dicts):
    script = []
    variables = {}
    for line_dict in script_dicts:
        line = f"WebUI.{line_dict['action']}({', '.join(line_dict['entities'])})"
        if line_dict['action'] == 'verifyMatch':
            actual = variables['getText']
            expected = actual.replace("actual", "expected")
            line = f"WebUI.{line_dict['action']}({actual}, {expected}, false)"

        if 'var_name' in line_dict:
            line = f"{line_dict['var_type']} {line_dict['var_name']} = {line}"
            variables[line_dict['action']] = line_dict['var_name']

        script.append(line)
    return '\n'.join(script)

### Test first 8 cases

In [15]:
test_cases = [
    [{'V': 'enter',
       'location': 'to email textbox',
       'value': 'admin1@mail.com',
       'page': 'login'},
      {'V': 'enter',
       'location': 'to password textbox',
       'value': 'Admin@123',
       'page': 'login'},
      {'V': 'click', 'value': 'Login', 'page': 'login'},
      {'V': 'wait',
       'value': 'title to be present for seconds',
       'time': 30,
       'page': 'login'}],
     [{'V': 'enter',
       'location': 'to email textbox',
       'value': 'invalid@wrong',
       'page': 'login'},
      {'V': 'enter',
       'location': 'to password textbox',
       'value': 'invalidpassword',
       'page': 'login'},
      {'V': 'click', 'value': 'Login', 'page': 'login'},
      {'V': 'wait',
       'value': 'email error message to appear for seconds',
       'time': 3,
       'page': 'login'},
      {'V': 'get', 'value': 'email error message', 'page': 'login'},
      {'V': 'check', 'page': 'login'},
      {'V': 'wait',
       'value': 'password error message to appear for seconds',
       'time': 3,
       'page': 'login'},
      {'V': 'get', 'value': 'password error message', 'page': 'login'},
      {'V': 'check', 'page': 'login'}],
     [{'V': 'click', 'value': 'the avatar', 'page': 'logout'},
      {'V': 'click', 'value': 'Sign out', 'page': 'logout'},
      {'V': 'wait', 'value': 'Login', 'time': 10, 'page': 'logout'}],
     [{'V': 'click', 'value': "user 's avatar", 'page': 'change password'},
      {'V': 'click', 'value': 'change password link', 'page': 'change password'},
      {'V': 'wait',
       'value': 'title to appear for s',
       'time': 10,
       'page': 'change password'},
      {'V': 'enter',
       'location': 'current password text box',
       'value': 'Admin@123',
       'page': 'change password'},
      {'V': 'enter',
       'location': 'new password text box',
       'value': 'Admin@1234',
       'page': 'change password'},
      {'V': 'enter',
       'location': 'to confirm password',
       'value': 'Admin@1234',
       'page': 'change password'},
      {'V': 'click', 'value': 'submit', 'page': 'change password'}],
#     [{'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'click', 'value': 'create library button', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'input',
#        'location': 'to the name field textbox',
#        'value': 'new library',
#        'page': 'Create Libraries'},
#       {'V': 'click', 'value': 'submit button', 'page': 'Create Libraries'}],
#      [{'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'click', 'value': 'Create Library', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'set', 'page': 'Create Libraries'},
#       {'V': 'click', 'value': 'Submit', 'page': 'Create Libraries'},
#       {'V': 'wait',
#        'value': 'for invalid message text to be present for s',
#        'time': 20,
#        'page': 'Create Libraries'},
#       {'V': 'get', 'value': 'invalid message text', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'}],
#      [{'V': 'wait',
#        'value': 'Add Library',
#        'time': 20,
#        'page': 'Create Libraries'},
#       {'V': 'click', 'value': 'Add Library', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'set', 'page': 'Create Libraries'},
#       {'V': 'click', 'value': 'Submit', 'page': 'Create Libraries'},
#       {'V': 'wait',
#        'value': 'for success message text to be present for s',
#        'time': 20,
#        'page': 'Create Libraries'},
#       {'V': 'get', 'value': 'success message text', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'}],
#      [{'V': 'click', 'value': 'Add Library', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'click', 'value': 'Submit', 'page': 'Create Libraries'},
#       {'V': 'wait',
#        'value': 'for invalid message text to be present for s',
#        'time': 20,
#        'page': 'Create Libraries'},
#       {'V': 'get', 'value': 'invalid message text', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'click', 'value': 'Cancel', 'page': 'Create Libraries'}],
#      [{'V': 'click',
#        'value': 'library navigation item',
#        'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'verify', 'page': 'Create Libraries'},
#       {'V': 'click',
#        'value': 'library navigation item',
#        'page': 'Create Libraries'}],
#      [{'V': 'click',
#        'value': 'library name with rely on name',
#        'page': 'Update Libraries'},
#       {'V': 'click',
#        'value': 'update option with rely on name',
#        'page': 'Update Libraries'},
#       {'V': 'verify', 'page': 'Update Libraries'},
#       {'V': 'click', 'value': 'submit button', 'page': 'Update Libraries'}]
]

In [16]:
scripts_dicts = [revert_to_script_dicts(test_case) for test_case in test_cases]
# scripts_dicts[1]

In [17]:
# align_one_line_to_script({'V': 'get', 'value': 'email error message', 'page': 'login'})
# get_obj_path({'V': 'get', 'value': 'email error message', 'page': 'login'})
# get_obj_path({'V': 'enter', 'location': 'to email textbox', 'value': 'admin1@mail.com', 'page': 'login'})

In [18]:
# scripts_dicts[0]

In [19]:
for i, script_dicts in enumerate(scripts_dicts):
    try:
        print("Script " + str(i)+'\n')
        print(revert_to_script(script_dicts))
        print("-"*100)
    except:
        print(i)
        print(script_dicts)

Script 0

WebUI.setText(findTestObject('Page_Login/tbx_Email'), 'admin1@mail.com')
WebUI.setText(findTestObject('Page_Login/tbx_Password'), 'Admin@123')
WebUI.click(findTestObject('Page_Login/btn_Login'))
WebUI.waitForElementPresent(findTestObject('Page_ChangePassword/txt_Title'), 30, FailureHandling.STOP_ON_FAILURE)
----------------------------------------------------------------------------------------------------
Script 1

WebUI.setText(findTestObject('Page_Login/tbx_Email'), 'invalid@wrong')
WebUI.setText(findTestObject('Page_Login/tbx_Password'), 'invalidpassword')
WebUI.click(findTestObject('Page_Login/btn_Login'))
WebUI.waitForElementPresent(findTestObject('Page_Login/txt_EmailErrorMessage'), 3, FailureHandling.STOP_ON_FAILURE)
def actualEmailErrorMessage = WebUI.getText(findTestObject('Page_Login/txt_EmailErrorMessage'))
WebUI.verifyMatch(actualEmailErrorMessage, expectedEmailErrorMessage, false)
WebUI.waitForElementPresent(findTestObject('Page_Login/txt_PasswordErrorMessage'),